In [2]:
from sklearn.model_selection import ParameterGrid

grid = {"n_neighbors": [3, 5, 7],
        "metric": ["Manhattan", "Euclidean"]}

#ParameterGrid(grid)
list(ParameterGrid(grid))

[{'metric': 'Manhattan', 'n_neighbors': 3},
 {'metric': 'Manhattan', 'n_neighbors': 5},
 {'metric': 'Manhattan', 'n_neighbors': 7},
 {'metric': 'Euclidean', 'n_neighbors': 3},
 {'metric': 'Euclidean', 'n_neighbors': 5},
 {'metric': 'Euclidean', 'n_neighbors': 7}]

In [3]:
# 함수의 입력으로 사전을 입력받기: **의 사용
def f(a, b):
    return a + b

input_f = {"a": 1, "b": 2}
#input_f = {"b": 2, "a": 1}
f(**input_f)

3

#### 그리드 서치 실습 예제


- 사용 모델: (1) k-최근접 이웃
    - n_neighbors (3, 5, 7)
    - metric (euclidean, manhattan)

- 사용 모델: (2) 서포트 벡터 머신
    - kernel: rbf, linear
    - C: 0.1, 1, 10

- 평가 척도: F1 score

In [4]:
from sklearn.datasets import load_iris
X = load_iris()['data'] # feature
Y = load_iris()['target'] # label

# 학습 데이터와 평가 데이터 분할
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [5]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC

# 파라미터 그리드 생성
param_grid = dict() 
# 입력: 모델 함수, 출력: 모델의 하이퍼 파라미터 그리드

# 모델별 파라미터 그리드 생성
param_grid_for_knn = ParameterGrid({"n_neighbors": [3, 5, 7],
                           "metric":['euclidean', 'manhattan']})

param_grid_for_svm = ParameterGrid({"C": [0.1, 1, 10],
                           "kernel":['rbf', 'linear']})

# 모델 - 하이퍼 파라미터 그리드를 param_grid에 추가
param_grid[KNN] = param_grid_for_knn
param_grid[SVC] = param_grid_for_svm

In [6]:
# 하이퍼 파라미터 튜닝 
best_score = -1 # 현재까지 찾은 가장 높은 f1_score (f1 score는 절대 0보다 작을수 없기에, -1로 설정해도 무방)

from sklearn.metrics import f1_score

for model_func in [KNN, SVC]:
    for param in param_grid[model_func]:
        model = model_func(**param).fit(Train_X, Train_Y) #사전이라 **
        pred_Y = model.predict(Test_X)        
        score = f1_score(Test_Y, pred_Y, average = 'micro') # 다중 분류일 때 macro, micro
        
        if score > best_score: 
            # 현재 점수가 지금까지 찾은 최고 점수보다 좋으면, 최고 모델, 파라미터, 점수 업데이트
            best_model_func = model_func
            best_score = score
            best_param = param

In [7]:
print("모델:", best_model_func)
print("점수:", best_score)
print("파라미터:", best_param)

모델: <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
점수: 0.9736842105263158
파라미터: {'metric': 'euclidean', 'n_neighbors': 3}


In [8]:
# 최종 모델 학습: 전체 X와 전체 Y에 대해.
final_model = best_model_func(**best_param).fit(X, Y)

### 데이터 크기에 따른 모델 선택

####  특징 개수가 매우 적은 경우

In [10]:
import os
os.chdir(r"C:\Users\Gyu\Desktop\mywork\FastOnline\Part 3. 지도학습 주요모델 및 개념\데이터")

import pandas as pd
df = pd.read_csv("Combined_Cycle_Power_Plant.csv")
df.shape

(9568, 5)

In [11]:
# 특징과 라벨 분리
X = df.drop('EP', axis = 1)
Y = df['EP']

In [12]:
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor as MLP
from sklearn.linear_model import LinearRegression as LR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.neighbors import KNeighborsRegressor as KNR

# 공정한 비교를 위해 전부 default 값을 사용
# random_state가 있는 모델은 모두 같은 값으로 설정

SVR_model = SVR()
MLP_model = MLP(random_state = 100)
LR_model = LR()
DTR_model = DTR(random_state = 100)
RFR_model = RFR(random_state = 100)
KNR_model = KNR()

model_list = [SVR_model, MLP_model, LR_model, DTR_model, RFR_model, KNR_model]
model_name_list = ['SVR', 'MLP', 'LR', 'DTR', 'RFR', 'KNR']

In [13]:
from sklearn.model_selection import cross_val_score
for (model, model_name) in zip(model_list, model_name_list):
    score = -cross_val_score(model, X, Y, cv = 5, scoring = 'neg_mean_absolute_error').mean() # -MAE이므로 다시 -를 붙인 것
    print(model_name, score)

SVR 11.15624063928403
MLP 3.9421874629947724
LR 3.628251380729045
DTR 3.0464405292720267
RFR 2.365834244467129
KNR 2.9176435395285303


변수 타입 확인 방법

In [16]:
df = pd.read_csv("Telco_churn_prediction.csv")
df.infer_objects().dtypes 

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [17]:
def find_str_element(val):
    try:
        float(val) # 만약 val이 문자라면 여기서 오류가 발생할 것이므로 except로 넘어감
        return False
    except:
        return True

# 공백이 섞여 있음을 확인
# apply 함수는 자주 사용되는 굉장히 중요한 함수이므로 반드시 숙지해야 함
# apply 함수는 각 요소에 함수를 일괄 적용하는 함수임
print(df['TotalCharges'][df['TotalCharges'].apply(find_str_element)].values)

[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']


In [19]:
def getting_unique_val(col):
    return len(col.unique())

df.apply(getting_unique_val, axis = 0) 
# 유니크한 값 개수 기준으로 판단
# 일반적으로 그 개수가 많으면 연속형, 그렇지 않으면 범주형인 경우가 대다수 (ID 관련 컬럼 제외)

customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64

혼합형 변수가 존재하는 경우
- 회귀, 나이브 베이즈, knn은 적합 하지 않은 편

---

### 복잡도 파라미터 튜닝

all model
- max_iter : 복잡한 모델을 학습할 때 일부러 작게 잡아 과적합을 회피하기도 함

정규화 회귀
- alpha : 복잡도와 반비례

의사 결정 나무
- max_depth : 복잡도와 정비례
- min_samples_leaf : 복잡도와 반비례

SVM
- C, gamma, degree : 복잡도와 약한 정비례
- kernel : poly > rbf > linear 순으로 과적합 가능성 

로지스틱
- C : 복잡도와 반비례

SVR
- epsilon : 복잡도와 강한 정비례

Tree Ensemble
- max_depth : 복잡도와 정비례지만 과적합 피하기 위해 보통 4이하 설정
- learning rate(rf 제외) : 복잡도와 정비례

<br>

초기값의 영향이 매우 크므로, **seed 고정**

In [21]:
df = pd.read_csv("Sonar_Mines_Rocks.csv")

In [22]:
X = df.drop('Y', axis = 1)
Y = df['Y']
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [23]:
Train_Y.replace({"M":-1, "R":1}, inplace = True)
Test_Y.replace({"M":-1, "R":1}, inplace = True)

Case 1. 복잡도 파라미터가 한 개이면서, 단순하고, 우연성이 어느정도 있는 모델 (Logistic Regression)

In [25]:
from sklearn.linear_model import LogisticRegression as LR

def LR_model_test(C):
    model = LR(C = C, max_iter = 100000, random_state = 10).fit(Train_X, Train_Y) # 가벼운 모델이므로 max_iter를 크게 잡음
    pred_Y = model.predict(Test_X)
    return f1_score(Test_Y, pred_Y)

print("C = 0.1:\t{}".format(LR_model_test(C = 0.1)))
print("C = 1:\t{}".format(LR_model_test(C = 1))) 
print("C = 5:\t{}".format(LR_model_test(C = 5)))

C = 0.1:	0.5581395348837209
C = 1:	0.7307692307692308
C = 5:	0.7692307692307692


In [28]:
# 아직 범위가 넓어 더 탐색
import numpy as np
LR_parameter_grid = ParameterGrid({"C":np.linspace(0.1, 2, 50),
                                  "max_iter":[100000],
                                  "random_state":[10]})

# 파라미터 튜닝 수행 
best_score = -1
for parameter in LR_parameter_grid:
    model = LR(**parameter).fit(Train_X, Train_Y)
    pred_Y = model.predict(Test_X)
    score = f1_score(Test_Y, pred_Y)
    
    if score > best_score:
        best_score = score
        best_parameter = parameter

print(best_parameter, best_score)

{'C': 1.6510204081632653, 'max_iter': 100000, 'random_state': 10} 0.7777777777777778


Case 2. 복잡도 파라미터가 두 개이면서, 단순하고, 우연성이 거의 없는 모델 (Decision Tree)

In [29]:
from sklearn.tree import DecisionTreeClassifier as DTC

def DTC_model_test(max_depth, min_samples_leaf):
    model = DTC(max_depth = max_depth, min_samples_leaf = min_samples_leaf).fit(Train_X, Train_Y) 
    pred_Y = model.predict(Test_X)
    return f1_score(Test_Y, pred_Y) 

for max_depth in [3, 6, 9]:
    for min_samples_leaf in [1, 2, 3]:
        score = DTC_model_test(max_depth = max_depth, min_samples_leaf = min_samples_leaf)
        print("{}-{}:{}".format(max_depth, min_samples_leaf, score))

# max depth가 크고 (복잡도 증가) min_samples_leaf가 큰 경우 (복잡도 감소) 좋은 성능이 나옴을 확인

3-1:0.7272727272727273
3-2:0.7272727272727273
3-3:0.7272727272727273
6-1:0.75
6-2:0.7169811320754716
6-3:0.6909090909090909
9-1:0.7796610169491525
9-2:0.7142857142857143
9-3:0.7037037037037037


In [30]:
# 파라미터 그리드 설정
DTC_parameter_grid = ParameterGrid({"max_depth": np.arange(6, 15),
                                  "min_samples_leaf": np.arange(2, 5)})

# 파라미터 튜닝 수행 
best_score = -1
for parameter in DTC_parameter_grid:
    model = DTC(**parameter).fit(Train_X, Train_Y)
    pred_Y = model.predict(Test_X)
    score = f1_score(Test_Y, pred_Y)
    
    if score > best_score:
        best_score = score
        best_parameter = parameter

print(best_parameter, best_score)

{'max_depth': 8, 'min_samples_leaf': 2} 0.7777777777777778
